## Obtaining Climate Data from Google Earth Engine

To investigate how climate impacts human rodent-borne disease cases, we must obtain historical climate data for a variety of countries. The climate parameters we are interested in and the datasets we are sourcing from:

Parameters used from other paper
Tmean, minimum daily air temperature—Tmin, and maximum daily air temperature—Tmax (all temperatures are the monthly averages of the corresponding daily values, in 2 m height above ground, in °C); total precipitation in mm—Pr, total sunshine duration in hours—SD, mean monthly soil temperature in 5 cm depth under uncovered typical soil of location in °C—ST, and soil moisture under grass and sandy loam in percent plant useable water—SM. 

WorldClim for historical data? https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD


|Parameter|Dataset|Resolution|Details|Reference|
|----|----|----|----|---|
|Mean daily land surface temperature|MODIS MOD11A1 v061|1 km|daily temperature|https://doi.org/10.5067/MODIS/MOD11A1.061|
|Monthly total precipitation|CHIRPS 2.0|0.05 degrees (5566 meters)|Climate hazards group InfraRed Precip wth station data. Each image represents a pentad (5 days).|hhttps://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD
|Soil Temperature|
|Enhaced vegetation index|MODIS Terra Vegetation 16-Day Global 1km|

In [495]:
# import packages
import ee
from datetime import datetime
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import geemap
import csv

Initialize Earth Engine. If first time using GEE in notebook setup, make sure that a project has been created and your account as been added, then run the authenticate command to link your account. Additionally, set up the working directory. Enter the authorization code you receive from google into the box. 

In [496]:
ee.Authenticate()
ee.Initialize()

wd = '~/Projects/RBDML'


Successfully saved authorization token.


### Define Regions of Interest

The case data for hantavirus and lassa fever only focus on the following countries based on the listed criteria. We will be downloading gridded raster for the climate parameters 

Country inclusion criteria:
- More than 10 cases in one month
- More than 300 total cases across all years

Countries included:

In [373]:
# import csv with regions and date ranges
df = pd.read_csv(os.path.join(wd, 'data/human/processed/country_list.csv'))
#df.set_index('country', inplace=True)
df

,country,FIPS,minyear,maxyear,max_monthly_cases,total_cases
0,Austria,AU,2008,2021,46,1134
1,Brazil,BR,2001,2020,66,577048
2,Chile,CI,1995,2022,19,9511
3,China,CH,2004,2023,4189,228738
4,Finland,FI,1995,2023,1596,2025954
5,France,FR,2011,2021,92,1357
6,Germany,GM,2001,2023,1639,33564817
7,Nigeria,NI,2012,2023,469,213120
8,Norway,NO,2008,2021,12,334
9,Slovakia,LO,2008,2021,34,470


### 1. Obtain Land Surface Temperature (LST)

First we will define the functions needed:

In [528]:
def scaleAndMask(img):
    '''
        Takes in an image and returns the the day LST in celcius, masked by quality.
    '''
    # Select the QA bands
    qc = img.select('QC_Day')

    # Create masks:
    qa_flag_mask = qc.bitwiseAnd(0b11).lt(2) # Bits 0-1 <= 1; good or other quality
    data_quality_mask = qc.bitwiseAnd(0b1100).rightShift(2).eq(0) # Bits 2-3 = 0; good data quality

    good_qc = qa_flag_mask.And(data_quality_mask)

    return(img.select('LST_Day_1km')
           .multiply(0.02) # convert from Kelvin
           .subtract(273.15)
           .updateMask(good_qc)
           .copyProperties(img, ['system:time_start']))

def eviMask(img):
    '''
        Takes in an image and returns the the day LST in celcius, masked by quality.
    '''
    # Select the QA bands
    qc = img.select('SummaryQA')

    # Create masks:
    qa_flag_mask = qc.bitwiseAnd(1 << 0).eq(0) # left-shift: only looking at last bit -> if 0 good quality

    return(img.select('EVI')
           .updateMask(qa_flag_mask)
           .copyProperties(img, ['system:time_start']))



def sumMonthlyComposite(collection, date):
    '''
        Calculate the sum for a month for an image.
    '''
    date = ee.Date(date)
    return (collection
            .filterDate(date, date.advance(1, 'month'))
            .sum()
            .set("month", date.format("MM"))
            .set("year", date.format("YYYY"))
            .set("system:index", date.format("MM-YYYY")))

def meanMonthlyComposite(collection, date):
    '''
        Calculate the mean for a month for an image.
    '''
    date = ee.Date(date)
    return (collection
            .filterDate(date, date.advance(1, 'month'))
            .mean()
            .set("month", date.format("MM"))
            .set("year", date.format("YYYY"))
            .set("system:index", date.format("MM-YYYY")))


def fcToDict(fc):
  '''
    Turns a feature collection into a dictionary. 
  '''
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

Next we need to create a list of the countries we will be using and load image collection.

In [358]:
# Feature Collection for list of countries
# Using the FAO GAUL Admin Layers: https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0#table-schema
# Export for admin 1 level for all 
#countries = ee.Filter.inList('ADM0_NAME', list(df['country'])[0]) # using FIPS country code
#regions = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(countries)

# Dates required
year = '2012'
start = ee.Date('2012-01-01')
end = ee.Date('2013-01-01')

# Image collection
# Import the MODIS land surface temperature collection.
#collection = ee.ImageCollection('MODIS/006/MOD11A1').filterDate(start,end).map(scaleAndMask)

### Export daily LST

In [490]:
collection = ee.ImageCollection('MODIS/006/MOD11A1').map(scaleAndMask)
collection_min_date = ee.Date(collection.aggregate_min('system:time_start'))
collection_max_date = ee.Date(collection.aggregate_max('system:time_start'))

for country in df['country']:
    # define date range
    start = ee.Date.fromYMD(df.loc[df['country'] == country, 'minyear'].item(), 1, 1)
    end = ee.Date.fromYMD(df.loc[df['country'] == country, 'maxyear'].item(), 12, 31)

    # if dates are outside of collection range
    if start.difference(collection_min_date, 'days').getInfo() < 0: start = collection_min_date
    if end.difference(collection_max_date, 'days').getInfo() > 0: end = collection_max_date
    
    region = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM0_NAME', country))
    daily_lst_filtered = collection.filterDate(start, end)

    # Determine mean for admin region
    daily_lst = daily_lst_filtered.map(lambda image: image.reduceRegions(
        reducer=ee.Reducer.mean().setOutputs(['mean_lst']),
        collection=region,
        scale=1000
    )).flatten()

    # Export the FeatureCollection to Google Drive as a CSV file
    task = ee.batch.Export.table.toDrive(
        collection=daily_lst,
        description=country+'_DailyLST',
        fileFormat='CSV',
        folder='test',
        selectors=['system:index', 'ADM0_CODE',	'ADM0_NAME', 'ADM1_CODE', 'ADM1_NAME', 'mean_lst']
    )

    # Start the export task
    task.start()
    print('Starting task for '+ country)

Starting task for Austria
Starting task for Brazil
Starting task for Chile
Starting task for China
Starting task for Finland
Starting task for France
Starting task for Germany
Starting task for Nigeria
Starting task for Norway
Starting task for Slovakia
Starting task for Slovenia
Starting task for Sweden
Starting task for United States of America


## Precipitation Data

In [532]:
collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
collection_min_date = ee.Date(collection.aggregate_min('system:time_start'))
collection_max_date = ee.Date(collection.aggregate_max('system:time_start'))

for country in ['Brazil', 'Chile']:
    # define date range
    start = ee.Date.fromYMD(df.loc[df['country'] == country, 'minyear'].item(), 1, 1)
    end = ee.Date.fromYMD(df.loc[df['country'] == country, 'maxyear'].item(), 12, 31)

    # if dates are outside of collection range
    if start.difference(collection_min_date, 'days').getInfo() < 0: 
        print("Data for "+country+" starts earlier than dataset.")
        start = collection_min_date
    if end.difference(collection_max_date, 'days').getInfo() > 0: 
        print("Data for "+country+" extends past dataset limit. Setting end date as ")
        end = collection_max_date
    
    region = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM0_NAME', country))
    filtered_collection = collection.filterDate(start, end)

    # determine mean for admin region
    daily_precip = filtered_collection.map(lambda image: image.reduceRegions(
        reducer=ee.Reducer.mean().setOutputs(['precipitation']),
        collection=region,
        scale=5000
    )).flatten()

    # Export the FeatureCollection to Google Drive as a CSV file
    task = ee.batch.Export.table.toDrive(
        collection=daily_precip,
        description=country+'_DailyCHIRPSPrecip',
        fileFormat='CSV',
        folder='test',
        selectors=['system:index', 'ADM0_CODE',	'ADM0_NAME', 'ADM1_CODE', 'ADM1_NAME', 'precipitation']
    )

    # Start the export task
    task.start()
    print('Starting task for '+ country)

Starting task for Brazil
Starting task for Chile


## MODIS EVI DATA

In [531]:
collection = ee.ImageCollection("MODIS/061/MOD13A2").map(eviMask)
collection_min_date = ee.Date(collection.aggregate_min('system:time_start'))
collection_max_date = ee.Date(collection.aggregate_max('system:time_start'))

for country in df['country']:
    # define date range
    start = ee.Date.fromYMD(df.loc[df['country'] == country, 'minyear'].item(), 1, 1)
    end = ee.Date.fromYMD(df.loc[df['country'] == country, 'maxyear'].item(), 12, 31)

    # if dates are outside of collection range
    if start.difference(collection_min_date, 'days').getInfo() < 0: 
        print("Data for "+country+" starts earlier than dataset.")
        start = collection_min_date
    if end.difference(collection_max_date, 'days').getInfo() > 0: 
        print("Data for "+country+" extends past dataset limit.")
        end = collection_max_date
    
    region = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM0_NAME', country))
    filtered_collection = collection.filterDate(start, end)

    # determine mean for admin region
    daily_precip = filtered_collection.map(lambda image: image.reduceRegions(
        reducer=ee.Reducer.mean().setOutputs(['EVI']),
        collection=region,
        scale=1000
    )).flatten()

    # Export the FeatureCollection to Google Drive as a CSV file
    task = ee.batch.Export.table.toDrive(
        collection=daily_precip,
        description=country+'_MonthlyModisEVI',
        fileFormat='CSV',
        folder='test',
        selectors=['system:index', 'ADM0_CODE',	'ADM0_NAME', 'ADM1_CODE', 'ADM1_NAME', 'EVI']
    )

    # Start the export task
    task.start()
    print('Starting task for '+ country)

Starting task for Austria
Starting task for Brazil
Data for Chile starts earlier than dataset.
Starting task for Chile
Data for China extends past dataset limit.
Starting task for China
Data for Finland starts earlier than dataset.
Data for Finland extends past dataset limit.
Starting task for Finland
Starting task for France
Data for Germany extends past dataset limit.
Starting task for Germany
Data for Nigeria extends past dataset limit.
Starting task for Nigeria
Starting task for Norway
Starting task for Slovakia
Starting task for Slovenia
Data for Sweden extends past dataset limit.
Starting task for Sweden
Data for United States of America starts earlier than dataset.
Starting task for United States of America
